# 이항변수화 (Continued)

전처리 할 때 범주형 변수를 이항변수화 시키는 작업을 많이 하게 된다.
* 범주형 변수의 예: male/female, 10대/20대/30대/... , A/B/C/D/F 
주의: 연속형 변수와 범주형 변수를 잘 구분해야 한다. 범주형 변수에 저장된 값은 ML이나 DL에서는 그대로 쓸 수는 없고 이항변수화 과정을 거쳐야 한다.
* 이항변수화 예:
    * 성별: 0 => 10, 1 => 01
    * 연령대: 10대(0) => 100000000, 20대(1) =>01000000 ...
이항변수화에는 one-hot encoder 라는 걸 사용한다

## OneHotEncoder

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [10]:
# 범주형 변수 [성별(1/0)], [연령대(0/1/2)], [성적(0/1/2/3/4)])
data = np.array([[0, 1, 1],     # 여성, 30대, B학점
                 [0, 2, 2],    # 여성, 40대, C학점
                 [1, 0, 3],    # 남성, 20대, D학점
                 [1, 1, 4],    # 남성, 30대, F학점
                 [0, 0, 0]])    # 여성, 20대, A학점
data


array([[0, 1, 1],
       [0, 2, 2],
       [1, 0, 3],
       [1, 1, 4],
       [0, 0, 0]])

In [11]:
ohe = OneHotEncoder()
ohe.fit(data)    # data 변수에 저장된 데이터에 맞춰서 이항변수화 시켰다.

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

### active_features_

In [12]:
ohe.active_features_

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

* array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) 의 의미는 [남, 여, 20대, 30대, 40대, A, B, C, D, E] 이다

### n_values_

In [13]:
ohe.n_values_

array([2, 3, 5])

* n_values_: 각 feature별 범주의 개수 출력

### feature_indices_

In [14]:
ohe.feature_indices_

array([ 0,  2,  5, 10], dtype=int32)

* 0, 2: 0이상 2미만
* 2, 5: 2이상 5미만
* 5, 10: 5이상 10미만

### 이항변수화

예) 여성, 30대, D학점 => (0, 1, 3)

0 -> 10, 1 -> 010, D -> 00010

In [15]:
test = np.array([[0, 1, 3]])
ohe.transform(test)    # transform 안에 2차원 array를 넣어야 한다

<1x10 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [16]:
ohe.transform(test).toarray()

array([[1., 0., 0., 1., 0., 0., 0., 0., 1., 0.]])

* 위의 결과를 끊어서 읽을 때 feature_indices_ 같은걸 쓴다

# 이산형화

이산형화: 연속형 변수를 범주형 변수로 변환하는 작업

np.digitize, np.where: 연속형 변수를 이산형화 (2개 이상)

## digitize

In [19]:
from pandas import DataFrame

In [22]:
np.random.seed(75)
df = DataFrame({
    "d1": np.random.randn(10),
    "d2": ["x", "x", "x", "x", "x", "y", "y", "y", "y", "y"]
})
df

,d1,d2
0,-0.709502,x
1,0.112694,x
2,0.477022,x
3,1.935981,x
4,0.450415,x
5,-1.188847,y
6,0.613631,y
7,-0.178142,y
8,1.346521,y
9,1.161150,y


In [24]:
df[["d1"]].min()    # ["d1"]으로 참조하면 Series로 출력되고, [["d1"]]으로 참조하면 DataFrame으로 출력된다.

d1   -1.188847
dtype: float64

d1 컬럼의 데이터를 5개의 범주로 나누고 싶다

In [29]:
bins = np.linspace(df.d1.min(), df.d1.max(), 5)    # 구간 나누기
bins    # 첫번째 구간: -1.1 ~ -0.4, 두번째 구간: -0.4 ~ 0.3, 세번째 구간 :

array([-1.18884736, -0.40764024,  0.37356688,  1.154774  ,  1.93598112])

In [31]:
np.digitize(df["d1"], bins)    # 각 값이 어떤 범주에 속하는지 출력    # 범주는 1~4고, 5는 최대값

array([1, 2, 3, 5, 3, 1, 3, 2, 4, 4], dtype=int64)

In [32]:
df["d1_bin"] = np.digitize(df["d1"], bins)
df

,d1,d2,d1_bin
0,-0.709502,x,1
1,0.112694,x,2
2,0.477022,x,3
3,1.935981,x,5
4,0.450415,x,3
5,-1.188847,y,1
6,0.613631,y,3
7,-0.178142,y,2
8,1.346521,y,4
9,1.161150,y,4


In [33]:
df.groupby("d1_bin")

In [34]:
df.groupby("d1_bin")["d1"].size()    # 그룹별로 나눴을 때 d1열의 값 개수

d1_bin
1    2
2    2
3    3
4    2
5    1
Name: d1, dtype: int64

In [35]:
df.groupby("d1_bin")["d1"].mean()    # 그룹별로 나눴을 때 d1값 평균

d1_bin
1   -0.949175
2   -0.032724
3    0.513689
4    1.253836
5    1.935981
Name: d1, dtype: float64

In [36]:
df.groupby("d1_bin")["d1"].sum()
df.groupby("d1_bin")["d1"].std()

d1_bin
1    0.338948
2    0.205652
3    0.087568
4    0.131077
5         NaN
Name: d1, dtype: float64

In [38]:
df.groupby("d1_bin")["d2"].value_counts()    # 그룹별로 d2열의 값 개수

d1_bin  d2
1       x     1
        y     1
2       x     1
        y     1
3       x     2
        y     1
4       y     2
5       x     1
Name: d2, dtype: int64

In [41]:
df[df["d1_bin"]==3]    # boolean 참조

,d1,d2,d1_bin
2,0.477022,x,3
4,0.450415,x,3
6,0.613631,y,3


In [42]:
df2 = df[df["d1_bin"]==3]

## np.where

np.where(조건, 참일 때 수행, 거짓일 때 수행)

np.where(조건, 참일 때 수행, np.where(조건, 참일 때 수행, 거짓일 때 수행))

In [43]:
df["d1"].mean()
# 평균보다 높고 낮은걸 구분하고 싶을 때
df["hl"] = np.where(df["d1"] >= df["d1"].mean(), "High", "Low")
df

,d1,d2,d1_bin,hl
0,-0.709502,x,1,Low
1,0.112694,x,2,Low
2,0.477022,x,3,High
3,1.935981,x,5,High
4,0.450415,x,3,High
5,-1.188847,y,1,Low
6,0.613631,y,3,High
7,-0.178142,y,2,Low
8,1.346521,y,4,High
9,1.161150,y,4,High


In [45]:
df.groupby("hl")["d1"].size()
df.groupby("hl")["d1"].mean()

hl
High    0.997453
Low    -0.490949
Name: d1, dtype: float64

In [51]:
Q1 = np.percentile(df["d1"], 25)    # 25% 지점
Q3 = np.percentile(df["d1"], 75)    # 75% 지점
# np.where를 사용, Q3보다 크거나 같으면 high, Q1보다 크거나 같으면 medium, 나머지는 low
df["hml"] = np.where(df["d1"] >= Q3, "High", np.where(df["d1"] >= Q1, "Medium", "Low"))
df

,d1,d2,d1_bin,hl,hml
0,-0.709502,x,1,Low,Low
1,0.112694,x,2,Low,Medium
2,0.477022,x,3,High,Medium
3,1.935981,x,5,High,High
4,0.450415,x,3,High,Medium
5,-1.188847,y,1,Low,Low
6,0.613631,y,3,High,Medium
7,-0.178142,y,2,Low,Low
8,1.346521,y,4,High,High
9,1.161150,y,4,High,High


# 다항변수화

Polynomial: 다항변수화 시킬 때 사용하는 함수

In [54]:
data = np.arange(6).reshape(3,2)
data

array([[0, 1],
       [2, 3],
       [4, 5]])

data를 2차항 변수화

x1, x2 => 1, x1, x2, x1^2, x1*x2, x2^2

data의 첫번째 열을 x1, 두번째 열을 x2라고 생각한다

In [56]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
print(data)
print(poly.fit_transform(data))

[[0 1]
 [2 3]
 [4 5]]
[[ 1.  0.  1.  0.  0.  1.]
 [ 1.  2.  3.  4.  6.  9.]
 [ 1.  4.  5. 16. 20. 25.]]


# 데이터 재구조화

데이터 재구조화 (피봇테이블, stack/unstack, 크로스테이블, melt, widetolong)

## 피봇(테이블)

In [58]:
df = DataFrame({
    "c_id": ["c1", "c1", "c1", "c2", "c2", "c2", "c3", "c3", "c3"],
    "p_id": ["p1", "p2", "p3", "p1", "p2", "p3", "p1", "p2", "p3"],
    "amount": [20, 10, 0, 30, 20 ,30 ,0, 5, 10]
})
df

,c_id,p_id,amount
0,c1,p1,20
1,c1,p2,10
2,c1,p3,0
3,c2,p1,30
4,c2,p2,20
5,c2,p3,30
6,c3,p1,0
7,c3,p2,5
8,c3,p3,10


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
c_id      9 non-null object
p_id      9 non-null object
amount    9 non-null int64
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes


In [60]:
df.pivot(index="c_id", columns="p_id", values="amount")

p_id,p1,p2,p3
c_id,,,
c1,20,10,0
c2,30,20,30
c3,0,5,10


In [61]:
df.pivot_table(index="c_id", columns="p_id", values="amount")

p_id,p1,p2,p3
c_id,,,
c1,20,10,0
c2,30,20,30
c3,0,5,10


pivot과 pivot table 함수의 차이

-> 둘 이상의 변수로 index를 삼고자 하는 경우 pivot은 에러가 나지만 pivot table은 잘 나온다.

-> 집계함수 (그룹단위 sum, mean등 함수) 사용가능 여부에 따라 다르다. pivot 사용 불가, pivot table 사용 가능.

In [62]:
df = DataFrame({
    "c_id": ["c1", "c1", "c1", "c2", "c2", "c2", "c3", "c3", "c3"],
    "p_id": ["p1", "p2", "p3", "p1", "p2", "p3", "p1", "p2", "p3"],
    "amount": [20, 10, 0, 30, 20 ,30 ,0, 5, 10],
    "reg": ["S", "S", "S", "S", "S", "S", "A", "A", "A"]
})
df

,c_id,p_id,amount,reg
0,c1,p1,20,S
1,c1,p2,10,S
2,c1,p3,0,S
3,c2,p1,30,S
4,c2,p2,20,S
5,c2,p3,30,S
6,c3,p1,0,A
7,c3,p2,5,A
8,c3,p3,10,A


In [64]:
df.pivot(index=["c_id", "reg"], columns="p_id", values="amount")     # 에러가 나온다

ValueError: Length of passed values is 9, index implies 2

In [65]:
df.pivot_table(index=["c_id", "reg"], columns="p_id", values="amount")

,p_id,p1,p2,p3
c_id,reg,,,
c1,S,20,10,0
c2,S,30,20,30
c3,A,0,5,10


In [68]:
df.pivot(index="reg", columns="p_id", values="amount")    # 에러가 난다.  reg는 2개인데 p_id의 값은 3개이기 때문에.

ValueError: Index contains duplicate entries, cannot reshape

In [69]:
df.pivot_table(index="reg", columns="p_id", values="amount")     # pivot table에서는 재구조화를 하게 되면 default로 mean함수가 적용된다.

p_id,p1,p2,p3
reg,,,
A,0,5,10
S,25,15,15


In [70]:
df.pivot_table(index="reg", columns="p_id", values="amount", aggfunc=np.mean)

p_id,p1,p2,p3
reg,,,
A,0,5,10
S,25,15,15


In [71]:
df.pivot_table(index="reg", columns="p_id", values="amount", aggfunc=np.sum)

p_id,p1,p2,p3
reg,,,
A,0,5,10
S,50,30,30


# 합치기 (Continued)

In [74]:
df1 = pd.read_csv("data/concat_1.csv")
df2 = pd.read_csv("data/concat_2.csv")
df3 = pd.read_csv("data/concat_3.csv")

In [76]:
row_concat = pd.concat([df1, df2, df3])
row_concat

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,a4,b4,c4,d4
1,a5,b5,c5,d5
2,a6,b6,c6,d6
3,a7,b7,c7,d7
0,a8,b8,c8,d8
1,a9,b9,c9,d9


In [77]:
# row_concat의 4번째 행 추출 -> index 번호가 3번
row_concat.iloc[3,]

A    a3
B    b3
C    c3
D    d3
Name: 3, dtype: object

In [78]:
# row_concat의 5번째 행 추출 -> index 번호가 0번
row_concat.iloc[4,]

A    a4
B    b4
C    c4
D    d4
Name: 0, dtype: object

* i(nteger)loc -> (눈에는 안보일 수 있지만) 순서대로 numbering된 인덱스를 사용

In [80]:
new_row_series = pd.Series(["n1", "n2", "n3", "n4"])
new_row_series

0    n1
1    n2
2    n3
3    n4
dtype: object

In [81]:
# df1에 new_row_series를 연결
pd.concat([df1, new_row_series])

,A,B,C,D,0
0,a0,b0,c0,d0,NaN
1,a1,b1,c1,d1,NaN
2,a2,b2,c2,d2,NaN
3,a3,b3,c3,d3,NaN
0,NaN,NaN,NaN,NaN,n1
1,NaN,NaN,NaN,NaN,n2
2,NaN,NaN,NaN,NaN,n3
3,NaN,NaN,NaN,NaN,n4


series를 dataframe에 연결할 때는, series를 dataframe으로 만든 후에 연결해줘야 정상적으로 처리된다.

In [83]:
new_row_df = pd.DataFrame([["n1", "n2", "n3", "n4"]], columns=["A", "B", "C", "D"])
new_row_df

,A,B,C,D
0,n1,n2,n3,n4


In [84]:
pd.concat([df1, new_row_df])

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4


시리즈를 데이터프레임의 행으로 연결할 수 없다 (시리즈에는 열 이름이 없기 때문이다).

그러므로 시리즈를 데이터프레임으로 만든 후에 연결한다.

append를 이용해서 연결

In [85]:
df1.append(new_row_df)

,A,B,C,D
0,a0,b0,c0,d0
1,a1,b1,c1,d1
2,a2,b2,c2,d2
3,a3,b3,c3,d3
0,n1,n2,n3,n4


concat은 2개 이상의 DF를 연결할 수 있지만 append는 한개의 DF만 연결할 수 있다.